### Import

In [1]:
import numpy as np
import pandas as pd 
from collections import Counter
import os
import glob
import copy
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

### Read every cutoffs from each method

In [2]:
datasets_1 = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/bootstrap/gmm/cutoffs/' + "/*."+'csv'))]
files_1 = [file.split(".")[0] for file in sorted(os.listdir('../results/bootstrap/gmm/cutoffs/'))]

datasets_2 = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/bootstrap/kmeans/cutoffs/' + "/*."+'csv'))]
files_2 = [file.split(".")[0] for file in sorted(os.listdir('../results/bootstrap/kmeans/cutoffs/'))]

datasets_3 = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/bootstrap/roc/cutoffs/' + "/*."+'csv'))]
files_3 = [file.split(".")[0] for file in sorted(os.listdir('../results/bootstrap/roc/cutoffs/'))]

datasets_4 = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/bootstrap/tertile/cutoffs/' + "/*."+'csv'))]
files_4 = [file.split(".")[0] for file in sorted(os.listdir('../results/bootstrap/tertile/cutoffs/'))]

datasets_5 = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/bootstrap/mean_sd/cutoffs/' + "/*."+'csv'))]
files_5 = [file.split(".")[0] for file in sorted(os.listdir('../results/bootstrap/mean_sd/cutoffs/'))]

In [3]:
# make a dictionary that contains all cutoffs as a dataframe
gmm = dict()
kmeans = dict()
roc = dict()
tertile = dict()
mean_sd = dict()

for file_, name_ in zip(datasets_1, files_1):
    gmm[name_] = file_
    
for file_, name_ in zip(datasets_2, files_2):
    kmeans[name_] = file_
    
for file_, name_ in zip(datasets_3, files_3):
    roc[name_] = file_
    
for file_, name_ in zip(datasets_4, files_4):
    tertile[name_] = file_
    
for file_, name_ in zip(datasets_5, files_5):
    mean_sd[name_] = file_

In [4]:
#rename columns before combining the dataframes 
for i in kmeans:
    for column in kmeans[i].columns:
        kmeans[i].rename(columns={column: column.split('_')[0]}, inplace=True)

### Preprocess the cutoffs
* make a dictionary with all the cutoffs
    * keys would be the cohort names
    * values would be a dataframe with all the cutoffs of CSF biomarkers

In [5]:
def cohort_based_cutoffs(method_dfs):
    """ """

#     biomarkers = method_dfs['gmm_cutoffs_0'].columns.to_list()
    cutpoints = {i: pd.DataFrame(index=list(range(1000)), 
                columns=method_dfs[list(method_dfs.keys())[0]].columns) 
                for i in method_dfs[list(method_dfs.keys())[0]].index}

    for bioma in cutpoints['ADNI'].columns:
        
        for cohort in method_dfs[list(method_dfs.keys())[0]].index:
            
            for i in method_dfs:
                cutpoints[cohort].loc[int(i.split('_')[-1]), bioma] = method_dfs[i].loc[cohort, bioma]
                
    return cutpoints

In [6]:
gmm_new = cohort_based_cutoffs(gmm)
kmeans_new = cohort_based_cutoffs(kmeans)
roc_new = cohort_based_cutoffs(roc)
tertile_new = cohort_based_cutoffs(tertile)
mean_sd_new = cohort_based_cutoffs(mean_sd)

### Calculate the confidence intervals of CSF cutoffs for each method 

In [66]:
# results = {i: pd.DataFrame(index=['GMM', 'ROC', 'K-Means', 'Tertile', 'Mean ±2 SD'], 
#            columns=gmm[list(gmm.keys())[0]].columns) 
#             for i in gmm[list(gmm.keys())[0]].index}

results = {i: pd.DataFrame(index=gmm[list(gmm.keys())[0]].index, 
           columns=gmm[list(gmm.keys())[0]].columns) 
            for i in ['GMM', 'K-Means', 'Tertile', 'ROC', 'Mean ±2 SD']}

results_percentage = {i: pd.DataFrame(index=gmm[list(gmm.keys())[0]].index, 
           columns=gmm[list(gmm.keys())[0]].columns) 
            for i in ['GMM', 'K-Means', 'Tertile', 'ROC', 'Mean ±2 SD']}

In [67]:
def mean_confidence_interval(methodologies, results, confidence=0.95):
    
    
    for method in methodologies:    
        datasets_ = methodologies[method]

        for i in datasets_:

            for col in datasets_[i].columns:
    #             data = datasets_[i][col].to_list()
                a = 1.0 * np.array(datasets_[i][col])
                n = len(a)
                m, se = np.nanmean(a, 0), stats.sem(a, nan_policy="omit")
                h = se * stats.t.ppf((1 + confidence) / 2., n-1)
#                 results[i].loc[method, col] = f'{round(m, 2)} [{round(m-h, 2)}, {round(m+h, 2)}]'
                results[method].loc[i, col] = f'{round(m, 2)} [{round(m-h, 2)}, {round(m+h, 2)}]'
                results_percentage[method].loc[i, col] = f'{round((((m+h) - (m-h))/m) * 100, 2)} %'

In [68]:
methodologies = {'GMM': gmm_new, 'K-Means': kmeans_new, 'ROC': roc_new, 'Tertile': tertile_new, 
                 'Mean ±2 SD': mean_sd_new}

In [69]:
mean_confidence_interval(methodologies, results)

In [70]:
pd.concat(results, axis=1).dropna().to_csv("../results/bootstrap/confidence_intervals.csv")

In [71]:
results = pd.concat(results, axis=1)

In [73]:
results

GMM                           \
                    A-beta 1-42 in CSF              pTau in CSF   
ADNI           986.93 [986.01, 987.85]     34.68 [34.57, 34.79]   
EPAD        1035.65 [1034.95, 1036.35]     27.78 [27.73, 27.83]   
AIBL           749.01 [743.23, 754.79]     87.82 [86.77, 88.87]   
ARWIBO         589.23 [584.97, 593.48]  155.03 [148.17, 161.89]   
EDSD             782.2 [776.11, 788.3]  119.22 [117.68, 120.77]   
PREVENT-AD   1187.4 [1158.64, 1216.16]     71.13 [70.26, 72.01]   
PharmaCog      792.94 [791.15, 794.74]     92.99 [92.71, 93.27]   
NACC_ELISA     594.13 [592.61, 595.66]     81.67 [80.86, 82.48]   
EMIF_ELISA     742.14 [741.23, 743.04]      82.08 [81.9, 82.27]   
NACC_XMAP      298.68 [296.97, 300.39]     60.84 [59.93, 61.75]   
EMIF_XMAP      386.88 [384.24, 389.51]      51.65 [51.19, 52.1]   
DOD-ADNI    1412.85 [1394.86, 1430.83]     26.57 [26.45, 26.69]   
JADNI          414.71 [412.51, 416.91]     61.13 [60.81, 61.46]   

                                                        K-Means  \
                        tTau in CSF          A-beta 1-42 in CSF   
ADNI        356.89 [356.05, 357.73]     972.58 [971.58, 973.58]   
EPAD        307.01 [306.62, 307.39]   1033.32 [1032.64, 1034.0]   
AIBL        604.49 [596.99, 611.99]     683.97 [681.22, 686.72]   
ARWIBO      531.55 [526.67, 536.44]     523.83 [521.71, 525.95]   
EDSD        584.84 [576.22, 593.47]       741.8 [736.1, 747.49]   
PREVENT-AD  466.54 [457.22, 475.85]  1146.42 [1144.08, 1148.75]   
PharmaCog   773.42 [750.81, 796.03]     762.16 [759.54, 764.78]   
NACC_ELISA  564.65 [552.43, 576.87]      586.06 [584.92, 587.2]   
EMIF_ELISA   526.56 [524.2, 528.92]      697.67 [695.23, 700.1]   
NACC_XMAP      94.54 [93.82, 95.27]     290.71 [288.43, 292.98]   
EMIF_XMAP   227.69 [220.84, 234.54]     363.26 [362.31, 364.21]   
DOD-ADNI    303.96 [302.25, 305.68]  1273.46 [1267.14, 1279.77]   
JADNI       133.87 [132.58, 135.16]     392.54 [390.65, 394.43]   

                                                           \
                     pTau in CSF              tTau in CSF   
ADNI        28.08 [28.04, 28.11]   288.2 [287.82, 288.58]   
EPAD        19.85 [19.82, 19.87]  223.71 [223.48, 223.94]   
AIBL        77.67 [76.96, 78.37]  530.63 [523.33, 537.94]   
ARWIBO      69.14 [68.89, 69.39]   432.09 [428.8, 435.38]   
EDSD        86.71 [86.01, 87.41]  547.57 [542.96, 552.18]   
PREVENT-AD  50.78 [50.55, 51.01]  308.78 [306.64, 310.93]   
PharmaCog    68.62 [68.03, 69.2]   466.9 [464.62, 469.18]   
NACC_ELISA  61.63 [61.42, 61.84]   492.07 [490.2, 493.94]   
EMIF_ELISA  64.87 [64.78, 64.95]  432.54 [431.71, 433.37]   
NACC_XMAP   40.15 [40.02, 40.28]      70.22 [70.0, 70.44]   
EMIF_XMAP   37.54 [37.44, 37.64]  128.91 [128.35, 129.47]   
DOD-ADNI    22.37 [22.26, 22.48]  249.78 [248.76, 250.79]   
JADNI       59.88 [59.63, 60.13]   128.97 [127.9, 130.04]   

                               Tertile                        \
                    A-beta 1-42 in CSF           pTau in CSF   
ADNI           833.25 [831.24, 835.27]   21.84 [21.79, 21.9]   
EPAD           884.48 [883.54, 885.43]  19.86 [19.83, 19.89]   
AIBL                               NaN                   NaN   
ARWIBO                             NaN                   NaN   
EDSD                               NaN                   NaN   
PREVENT-AD  1098.71 [1096.81, 1100.61]   53.0 [52.86, 53.14]   
PharmaCog                          NaN                   NaN   
NACC_ELISA     625.54 [623.58, 627.49]  48.25 [48.12, 48.39]   
EMIF_ELISA     540.96 [539.73, 542.18]  52.28 [52.18, 52.39]   
NACC_XMAP      248.47 [247.68, 249.25]  36.49 [36.42, 36.56]   
EMIF_XMAP      446.65 [444.93, 448.38]  31.38 [31.13, 31.62]   
DOD-ADNI       928.41 [924.45, 932.38]   19.83 [19.77, 19.9]   
JADNI           392.71 [390.9, 394.52]  38.54 [38.46, 38.63]   

                                                         ROC  \
                        tTau in CSF       A-beta 1-42 in CSF   
ADNI        240.07

In [72]:
pd.concat(results_percentage, axis=1)

GMM                                    K-Means  \
           A-beta 1-42 in CSF pTau in CSF tTau in CSF A-beta 1-42 in CSF   
ADNI                   0.19 %      0.64 %      0.47 %             0.21 %   
EPAD                   0.14 %      0.33 %      0.25 %             0.13 %   
AIBL                   1.54 %      2.39 %      2.48 %              0.8 %   
ARWIBO                 1.44 %      8.85 %      1.84 %             0.81 %   
EDSD                   1.56 %      2.59 %      2.95 %             1.54 %   
PREVENT-AD             4.84 %      2.46 %      3.99 %             0.41 %   
PharmaCog              0.45 %      0.61 %      5.85 %             0.69 %   
NACC_ELISA             0.51 %      1.99 %      4.33 %             0.39 %   
EMIF_ELISA             0.24 %      0.46 %       0.9 %              0.7 %   
NACC_XMAP              1.15 %      2.99 %      1.54 %             1.56 %   
EMIF_XMAP              1.36 %      1.77 %      6.02 %             0.52 %   
DOD-ADNI               2.55 %       0.9 %      1.13 %             0.99 %   
JADNI                  1.06 %      1.05 %      1.93 %             0.96 %   

                                              Tertile                          \
           pTau in CSF tTau in CSF A-beta 1-42 in CSF pTau in CSF tTau in CSF   
ADNI            0.25 %      0.26 %             0.48 %       0.5 %      0.48 %   
EPAD            0.22 %       0.2 %             0.21 %      0.28 %      0.27 %   
AIBL            1.81 %      2.75 %                NaN         NaN         NaN   
ARWIBO          0.72 %      1.52 %                NaN         NaN         NaN   
EDSD            1.61 %      1.68 %                NaN         NaN         NaN   
PREVENT-AD       0.9 %      1.39 %             0.35 %      0.52 %      0.51 %   
PharmaCog       1.71 %      0.98 %                NaN         NaN         NaN   
NACC_ELISA      0.69 %      0.76 %             0.62 %      0.55 %       0.9 %   
EMIF_ELISA      0.27 %      0.38 %             0.45 %      0.39 %      0.57 %   
NACC_XMAP       0.64 %      0.64 %             0.63 %      0.38 %      0.61 %   
EMIF_XMAP       0.53 %      0.87 %             0.77 %      1.54 %      1.37 %   
DOD-ADNI        0.97 %      0.81 %             0.85 %      0.65 %      0.62 %   
JADNI           0.82 %      1.66 %             0.92 %      0.45 %       0.7 %   

                          ROC                                 Mean ±2 SD  \
           A-beta 1-42 in CSF pTau in CSF tTau in CSF A-beta 1-42 in CSF   
ADNI                   0.89 %      0.85 %      0.77 %              1.4 %   
EPAD                   1.23 %      3.02 %      3.39 %             0.54 %   
AIBL                      NaN         NaN         NaN                NaN   
ARWIBO                    NaN         NaN         NaN                NaN   
EDSD                      NaN         NaN         NaN                NaN   
PREVENT-AD                NaN         NaN         NaN             1.33 %   
PharmaCog                 NaN         NaN         NaN                NaN   
NACC_ELISA             1.47 %      1.16 %      1.82 %             1.22 %   
EMIF_ELISA             1.51 %      1.51 %      1.22 %             1.78 %   
NACC_XMAP              0.48 %      0.92 %       1.8 %             8.67 %   
EMIF_XMAP              1.14 %      1.71 %      1.14 %             2.45 %   
DOD-ADNI                  NaN         NaN         NaN             2.66 %   
JADNI                  0.85 %      1.09 %      0.41 %             1.84 %   

                                    
           pTau in CSF tTau in CSF  
ADNI            0.53 %      0.45 %  
EPAD            0.43 %      0.34 %  
AIBL               NaN         NaN  
ARWIBO             NaN         NaN  
EDSD               NaN         NaN  
PREVENT-AD      0.87 %      1.29 %  
PharmaCog          NaN         NaN  
NACC_ELISA      1.11 %      0.93 %  
EMIF_ELISA      1.08 %      2.56 %  
NACC_XMAP       1.19 %      0.97 %  
EMIF_XMAP       1.38 %      0.84 %  
DOD-ADNI        0.84 %      0.67 %  
JADNI     

In [53]:
results_percentage['GMM']

,A-beta 1-42 in CSF,pTau in CSF,tTau in CSF
ADNI,0.19,0.64,0.47
EPAD,0.14,0.33,0.25
AIBL,1.54,2.39,2.48
ARWIBO,1.44,8.85,1.84
EDSD,1.56,2.59,2.95
PREVENT-AD,4.84,2.46,3.99
PharmaCog,0.45,0.61,5.85
NACC_ELISA,0.51,1.99,4.33
EMIF_ELISA,0.24,0.46,0.9
NACC_XMAP,1.15,2.99,1.54
